# Lattices

In [ ]:
%matplotlib inline
import openmc
import numpy as np
from IPython.display import Image

In this section, we will build one of the assemblies from the BEAVRS benchmark.  This is a PWR assembly with fuel pins, guide tubes, and borosilicate glass burnable poisons.  A diagram of th assembly is plotted below.  To make it a little easier (and improve our statistics!) we will only build one quarter of it.

In [ ]:
Image('assembly_diagram.png')

## Materials

Again we have UO2, Zr, and H2O.  We also have borosilicate glass (pyrex).  This time we'll use the shortcut for defining enriched uranium.

In [ ]:
uo2 = openmc.Material(name='uo2')
uo2.add_element('U', 1.0, enrichment=3.0)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)

zirconium = openmc.Material(name='zirconium')
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name='water')
water.add_nuclide('H1', 2)
water.add_nuclide('O16', 1)
water.set_density('g/cm3', 0.7)
water.add_s_alpha_beta('c_H_in_H2O')

pyrex = openmc.Material(name='pyrex')
pyrex.add_element('B', 0.49)
pyrex.add_element('O', 4.7)
pyrex.add_element('Al', 0.17)
pyrex.add_element('Si', 1.8)
pyrex.set_density('g/cm3', 2.26)

In [ ]:
mf = openmc.Materials((uo2, zirconium, water, pyrex))
mf.export_to_xml()

Later in this example, we will make a bunch of geometry plots.  By default, every region is colored randomly and the results are Now that we know our materials, let's define a "color specification" to use when plotting our geometry.

In [ ]:
colors = {}
colors[water] = 'lightblue'
colors[zirconium] = 'gray'
colors[pyrex] = 'green'
colors[uo2] = 'red'

## Fuel pin

This is similar to the pincell example, but we don't have boundary conditions.  This `fuel_pin` universe extends to infinity

In [ ]:
pitch = 1.26

fuel_or = openmc.ZCylinder(R=0.39)
clad_ir = openmc.ZCylinder(R=0.40)
clad_or = openmc.ZCylinder(R=0.46)

fuel = openmc.Cell(name='fuel', fill=uo2)
fuel.region = -fuel_or

gap = openmc.Cell(name='air gap')
gap.region = +fuel_or & -clad_ir

clad = openmc.Cell(name='clad', fill=zirconium)
clad.region = +clad_ir & -clad_or

moderator = openmc.Cell(name='moderator', fill=water)
moderator.region = +clad_or

fuel_pin = openmc.Universe(cells=(fuel, gap, clad, moderator))

When building a complex geometry, it is helpful to plot each universe as you go along.  Let's plot this pincell now

In [ ]:
fuel_pin.plot(width=(pitch, pitch), color_by='material',
              colors=colors)

## Guide tube

Below you should build a guide tube universe. The guide tube has the following specs:

- Clad IR = 0.56 cm
- Clad OR = 0.60 cm
- Inside the clad is water
- Outside the clad is water

## Pyrex burnable poison

Now you need to model the burnable poison pin. It has the following specs:

- R < 0.21 cm, void
- 0.21 cm < R < 0.23 cm, zirconium
- 0.23 cm < R < 0.24 cm, void
- 0.24 cm < R < 0.43 cm, pyrex
- 0.43 cm < R < 0.44 cm, void
- 0.44 cm < R < 0.48 cm, zirconium
- 0.48 cm < R < 0.56 cm, water
- 0.56 cm < R < 0.60 cm, zirconium
- 0.60 cm < R, water

## The BEAVRS assembly

In [ ]:
Image('assembly_diagram.png')

Now that we have the universes needed, let's build the BEAVRS assembly.

We've already set up a `Settings` object below.

In [ ]:
settings = openmc.Settings()

lower_left, upper_right = main.region.bounding_box
spatial_dist = openmc.stats.Box(lower_left, upper_right)
settings.source = openmc.Source(space=spatial_dist)
settings.batches = 50
settings.inactive = 10
settings.particles = 1000
settings.verbosity = 5
settings.export_to_xml()

In [ ]:
openmc.run()

## Tallies

Okay, that was cool, but $k_\text{eff}$ isn't everyting.  We also want to know reaction rates so we can compute the power distribution, depletion rate, etc.  If we want pin-by-pin reaction rates, we have two options.  First, we can use a tally mesh.  This will lay a rectangular grid over the geometry and tally the reaction rates in each mesh bin.

Or we can use something called a "distribcell" filter.  Note that the bin specifies a cell id.

In [ ]:
openmc.run(output=False)

In [ ]:
!cat tallies.out | head -n 10
!cat tallies.out | tail -n 10

## Distributed materials

In a depletion problem, every fuel pin might need its own unique material.  We have a feature called "distributed materials" which makes this easier. The idea is that we can assign a different material to each unique instance of the cell. The first thing we want to know is how many instances of a particular cell there are. We first need to call the `Geometry.determine_paths()` method which determines all the unique paths to a particular cell. 

The order of the unique instances in the `paths` attribute also indicates the order in which we should specify materials. To specify a different material for each instance, we assign a list of materials to `fuel.fill` that is as long as the number of instances. Let's create a unique copy of UO2 for each instance and assign it to the fuel cell.

## Hexagonal Lattices

OpenMC also permits hexagonal lattices. They are a little trickier, but as we'll see there are some helper methods that demystify how to assign universes. We need to set the `center` of the lattice, the `pitch`, an `outer` universe (which is applied to all lattice elements outside of those that are defined), and a list of `universes`. Let's start with the easy ones first. Note that for a 2D lattice, we only need to specify a single number for the pitch.

### Rotating the lattice

Now let's say we want our hexagonal lattice orientated such that flat sides are parallel to the y-axis instead of the x-axis. This can be achieved by rotating the cell that contains the lattice by 30 degrees. Ideally, we'd specify a rotation on `main_cell`. However, we can only rotate a cell that is filled by a universe (not a lattice). Therefore, we'll create a dummy cell/universe that is filled with the lattice and change our main cell to be filled with that universe in order to rotate it. 